## Define Variables / Import MetaData

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import hvplot.xarray
import holoviews as hv
from cartopy import crs as ccrs 
import datetime
import glob
import os

In [ ]:
# Define misc variables
amer_filepath = '../ameriflux-data/'
mic_filepath = '../intermediates/'
timedelta = 'DD'

In [ ]:
# Import site metadata csv
meta_file = amer_filepath + 'AmeriFlux-site-search-results-202410071335.tsv'
ameriflux_meta = pd.read_csv(meta_file, sep='\t')
fluxnet_meta = ameriflux_meta.loc[ameriflux_meta['AmeriFlux FLUXNET Data'] == 'Yes'] #use FLUXNET only

In [ ]:
total_sites = fluxnet_meta['Site ID'].to_list()
len(total_sites)

In [ ]:
# set map proj
proj=ccrs.PlateCarree()

In [ ]:
def get_single_match(pattern):
    matches = glob.glob(pattern)
    if len(matches) == 1:
        return matches[0]
    elif len(matches) == 0:
        raise ValueError(f"No matches found")
    else:
        raise ValueError(f"Multiple matches found: {matches}")

In [ ]:
def get_amer_match(amer_filepath, site_ID):
    match = get_single_match(amer_filepath + 'AMF_' + site_ID +
                            '_FLUXNET_SUBSET_*/AMF_' + site_ID +
                            '_FLUXNET_SUBSET_' + timedelta + '*.csv')
    return match

## B. Develop script to dump all variables requested into one preprocessed file

In [ ]:
site_ID = 'AR-TF1'
# Define micasa variables wanted
micasa_var_list = ['NEE', 'NPP']
# Extract site lat/lon
site_lat = ameriflux_meta.loc[ameriflux_meta['Site ID'] == site_ID, 'Latitude (degrees)'].values
site_lon = ameriflux_meta.loc[ameriflux_meta['Site ID'] == site_ID, 'Longitude (degrees)'].values

In [ ]:
# Import fluxnet 
fluxnet_sel = pd.read_csv(get_amer_match(amer_filepath, site_ID))

In [ ]:
fluxnet_sel_dates = fluxnet_sel.loc[:,['TIMESTAMP']].copy()
fluxnet_sel_dates['TIMESTAMP'] = pd.to_datetime(fluxnet_sel_dates['TIMESTAMP'],format='%Y%m%d')
fluxnet_sel_dates = fluxnet_sel_dates.set_index('TIMESTAMP')
time = fluxnet_sel_dates.index
dates_unique = list({dt.date() for dt in time})
dates_unique.sort()
dates_unique[:5];

In [ ]:
# Import micasa
# Test some random files
path_test = glob.glob('/discover/nobackup/hzafar/ghgc/micasa/micasa-data/daily-0.1deg-final/holding/daily/2016/01/MiCASA_v1_flux_x3600_y1800_daily_2016010*')

In [ ]:
ds_out = pd.DataFrame()
with xr.open_mfdataset(path_test)[micasa_var_list] as ds:
    # Select grid closest to selected site
    ds_subset = ds.sel(lon=site_lon, lat=site_lat, method='nearest')
    # print(ds_subset)
    
    # Prep data for writing to csv
    ds_subset = ds_subset.squeeze(dim=['lat','lon'],drop=True)
    # print(ds_subset)
    
    # Output a single file for each site with all variables
    for micasa_var in micasa_var_list:
        ds_out[micasa_var] = ds_subset[micasa_var].to_dataframe()
        ds_out.rename(columns={micasa_var: f'MiCASA {micasa_var} ({ds_subset[micasa_var].units})'}, inplace=True)

In [ ]:
ds_out

In [ ]:
# !ls ../landmask

In [ ]:
# Import landmask file for 2001
landmask_filepath = '../landmask/'
year = str(2001)
ds = xr.open_dataset(landmask_filepath + 'MiCASA_v1_cover_x3600_y1800_yearly_' + year + '.nc4')

In [ ]:
ds

In [ ]:
ds.ftype

In [ ]:
ds.fbare.plot()

In [ ]:
# fig, ax = plt.subplots(subplot_kw={'projection': proj}) # Create a Cartopy projection and axis to fix error
# ds.fbare.plot(ax=ax, transform=ccrs.PlateCarree())

In [ ]:
ds.ftype.sel(type=17).plot()

In [ ]:
ds.ftype.hvplot(x='lat',y='lon', 
                crs=proj,
               size=150)